### Graph Class

In [14]:
from heapq import heappush, heappop 
import numpy
from numpy import inf

class Graph:

    def __init__(self, directed = False,  weighted = False):

        self.directed = directed
        self.weighted = weighted
        self.graph = {}
        self.vertex_num = 0

    def add_vertex(self, vertex):
        if not vertex in self.graph.keys():
            self.graph[vertex] = {}
            self.vertex_num += 1
        
    def add_edge(self, origin, destiny, weight=0):
        
        if not origin in self.graph.keys():
            self.add_vertex(origin)
            
        if not destiny in self.graph.keys():
            self.add_vertex(destiny)

        self.graph[origin][destiny] = weight
        if not self.directed:
            self.graph[destiny][origin] = weight
            
            
    def get_cost_matrix(self):
        
        graph = self.graph
        Cost = {}
        for i in graph.keys():
            Cost[i] = {}
            for j in graph.keys():
                if j in graph[i].keys():
                    Cost[i][j] = graph[i][j]
                else:
                    Cost[i][j] = 0 if i==j else inf
        return Cost
                   
        
    
    def SSSP(self, source, get_paths = False, algorithm = 'dijkstra'):
        '''
            Single source shortest path
        '''
        return self._dijkstra(source, get_paths = get_paths)
    
    
    
    def APSP(self, algorithm = 'floyd warshall'):
        '''
            All pairs shortest path
        '''
        
        if algorithm == 'floyd warshall':
            return self._floyd_warshall()
        
        if algorithm == 'dijkstra':
            Cost = {}
            for i in self.graph.keys():
                Cost[i] = self._dijkstra(i)
            return Cost
        
        raise Exception("Algorithm mispelled or not available")
        
        
        
    def _floyd_warshall(self):
        
        '''
            Floyd Warsall algorithm to solve the all pairs shortest path problem.
            Complexity: O(V^3)

            Output:
                distances : (Cost) Dictionary that, for every node, has the 
                                   shortest distance from that node to all the 
                                   other nodes in the graph
        
        '''
        
        graph = self.graph
        Cost = self.get_cost_matrix()
                        
        for i in graph.keys():
            for j in graph.keys():
                for k in graph.keys():
                    Cost[j][k] = min(Cost[j][k], Cost[j][i] + Cost[i][k])
                    
        return Cost   
        
        
        
    def _dijkstra(self, source, get_paths = False):
        '''
            Dijkstra algorithm (lazy deletion and using a heap to get the minimum)
            Complexity: O((V+E) log V)

            Input:
                source    : (Obj)  The source vertex

            Output:
                distances : (dict) Dictionary that, for every key, has the 
                                   shortest distance from the source to 
                                   that key value

                paths     : (dict) Dictionary that, for every key, has the an 
                                   array with the vertexes visited in the shortest 
                                   path from the source to that key

        '''
        graph = self.graph
        # The heap to obtain the minimum in each iteration
        heap = []

        distances = { source: 0 }
        paths = { source: [source] }

        # Updating heap
        for u in graph[source]:
            heappush(heap, (graph[source][u], u, source))

        while heap:
            # Vertex at the shortest distance in current step
            distance, s, last_vertex = heappop(heap)

            # Lazy deletion
            if s in distances.keys():
                continue

            # Obtained shortest distance to current vertex, updating values
            distances[s] = distance
            
            if get_paths:
                paths[s] = [v for v in paths[last_vertex]]
                paths[s].append(s)

            # Updating heap
            for u in graph[s]:
                if not u in distances.keys():
                    heappush(heap, (graph[s][u] + distances[s], u, s))

        if get_paths:
            return distances, paths
        else:
            return distances

### Example

In [15]:
# Example

G = Graph(weighted = True)

G.add_edge('a','b', 2)
G.add_edge('a','c', 4)
G.add_edge('b','c', 1)
G.add_edge('b','e', 5)
G.add_edge('b','d', 4)
G.add_edge('c','d', 2)
G.add_edge('c','f', 6)
G.add_edge('d','e', 1)
G.add_edge('d','f', 3)
G.add_edge('e','g', 4)

G.add_edge('f','g', 1)

graph = G.graph

In [22]:
G.SSSP('c')

{'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6}

In [21]:
G.SSSP('c', get_paths = True)

({'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6},
 {'a': ['c', 'b', 'a'],
  'b': ['c', 'b'],
  'c': ['c'],
  'd': ['c', 'd'],
  'e': ['c', 'd', 'e'],
  'f': ['c', 'd', 'f'],
  'g': ['c', 'd', 'f', 'g']})

In [16]:
G.APSP(algorithm = 'floyd warshall')

{'a': {'a': 0, 'b': 2, 'c': 3, 'd': 5, 'e': 6, 'f': 8, 'g': 9},
 'b': {'a': 2, 'b': 0, 'c': 1, 'd': 3, 'e': 4, 'f': 6, 'g': 7},
 'c': {'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6},
 'd': {'a': 5, 'b': 3, 'c': 2, 'd': 0, 'e': 1, 'f': 3, 'g': 4},
 'e': {'a': 6, 'b': 4, 'c': 3, 'd': 1, 'e': 0, 'f': 4, 'g': 4},
 'f': {'a': 8, 'b': 6, 'c': 5, 'd': 3, 'e': 4, 'f': 0, 'g': 1},
 'g': {'a': 9, 'b': 7, 'c': 6, 'd': 4, 'e': 4, 'f': 1, 'g': 0}}

In [17]:
G.APSP(algorithm = 'dijkstra' )

{'a': {'a': 0, 'b': 2, 'c': 3, 'd': 5, 'e': 6, 'f': 8, 'g': 9},
 'b': {'a': 2, 'b': 0, 'c': 1, 'd': 3, 'e': 4, 'f': 6, 'g': 7},
 'c': {'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6},
 'd': {'a': 5, 'b': 3, 'c': 2, 'd': 0, 'e': 1, 'f': 3, 'g': 4},
 'e': {'a': 6, 'b': 4, 'c': 3, 'd': 1, 'e': 0, 'f': 4, 'g': 4},
 'f': {'a': 8, 'b': 6, 'c': 5, 'd': 3, 'e': 4, 'f': 0, 'g': 1},
 'g': {'a': 9, 'b': 7, 'c': 6, 'd': 4, 'e': 4, 'f': 1, 'g': 0}}